In [1]:
import cv2
from math import sqrt
import numpy as np 
from matplotlib import pyplot as plt

In [2]:
path = "/home/ben/Code/Flow-Free-Solver-Using-OpenCV/Flow Free Game/6x6.jpg"
height = 500
width = 500

img = cv2.imread(path)
img = cv2.resize(img,(width, height))
imgBlank = np.zeros((height,width, 3), np.uint)

In [3]:
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
_, th4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)

res = cv2.bitwise_xor(img, th4)
imgGray_grid = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray_grid,(5,5),1)
imgThreshold = cv2.adaptiveThreshold(imgBlur,255,1,1,11,2)

imgGray_box = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur_box = cv2.GaussianBlur(imgGray_box,(5,5),1)
imgThreshold_box = cv2.adaptiveThreshold(imgBlur_box,255,1,1,11,2)

contours_box, heirarchy = cv2.findContours(imgThreshold_box, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [4]:
def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for cnt in contours:
        area= cv2.contourArea(cnt)
        if area>500:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri,True)
            if area>max_area and len(approx)==4:
                max_area = area
                biggest = approx
    return biggest, max_area

def reorder(points):
    points = points.reshape((4,2))
    pointsnew = np.zeros((4,1,2),dtype= np.int32)
    add = points.sum(1)
    pointsnew[0] = points[np.argmin(add)]
    pointsnew[3] = points[np.argmax(add)]
    diff = np.diff(points,axis=1)
    pointsnew[1] = points[np.argmin(diff)]
    pointsnew[2] = points[np.argmax(diff)]
    return pointsnew

def splitBoxes(img,n):
    rows = np.vsplit(img,6)
    boxes=[]
    for r in rows:
        cols = np.hsplit(r,6)
        for box in cols:
            boxes.append(box)
    return boxes

col_val = [[0,0,254,1],[0,141,2,2],[4,224,234,3],[254,41,14,4],[0,137,250,5]]

def predict(boxes):
    board = []
    for image in boxes:
        col = (image[50,50])
        e = 0
        for i in col_val:
            if i[0]==col[0] and i[1]==col[1] and i[2]==col[2]:
                e = i[3]
        board.append(e)
    return board
  

In [5]:
biggest_box, maxArea_box = biggestContour(contours_box)
imgBigcontours_box = img.copy()
if biggest_box.size!= 0:
    biggest = reorder(biggest_box)
    cv2.drawContours(imgBigcontours_box, biggest, -1,(0,0,255),10)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)


In [6]:
edges= cv2.Canny(imgGray_grid, 175,300)
contours, hierarchy= cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

biggest, maxArea = biggestContour(contours)

if biggest.size!= 0:
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    cv2.drawContours(img.copy(), biggest, -1,(0,0,255),10)
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (width,height))
    imgDetectedCircles = imgBlank.copy()
    # imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)

cv2.drawContours(img,contours, -1,(0,255,0),3)
cv2.imshow("Result", img)


In [7]:
imgSolvedDigits = imgBlank.copy()
n = 6
boxes = splitBoxes(imgWarpColored,n)
board = predict(boxes)
print(board)
bt  = open('board.txt','w')
bt.write(str(n)+"\n")
for i in range(len(board)):
    bt.write(str(board[i])+" ")
bt.close()


ValueError: array split does not result in an equal division

: 

In [ ]:
#  cv2.imshow("Original", img)
#  cv2.imshow("4 - THRESH_TOZERO", th4)
#  cv2.imshow("Result", res)
#  cv2.imshow("Grid", img)
#  cv2.imshow("Threshold", imgThreshold)
#  cv2.imshow("Gridbox", imgBigcontours_box)
# cv2.imshow("Image Grid", imgGray_grid)

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()